In [1]:
import os

import timm
import numpy as np
import pandas as pd
import cv2
import torch
import fastbook
import fastai
import fastcore
import PIL
import shutil
import albumentations

import matplotlib.pyplot as plt
import imgaug as ia
import seaborn as sns

from imgaug import augmenters as iaa
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from fastai.vision.all import *
from nbdev.showdoc import *
from fastai.vision.all import *
import torch.utils.mobile_optimizer as mobile_optimizer
from torch.utils.mobile_optimizer import optimize_for_mobile

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.8 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


In [11]:
learn_inference = torch.jit.load("bestISICFocal_512_32_50.pt")


/home/cristhian/anaconda3/lib/python3.11/site-packages/torch/serialization.py:1006: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


In [13]:
scripted_module = torch.jit.script(learn_inference)
optimized_scripted_module = optimize_for_mobile(scripted_module)

# using optimized lite interpreter model makes inference about 60% faster than the non-optimized lite interpreter model, which is about 6% faster than the non-optimized full jit model
scripted_module._save_for_lite_interpreter("my_model_lite.ptl")

In [21]:
valid_data = pd.read_csv("valid.csv")
android_data = pd.read_csv("validandroid.csv")

valid_data = valid_data.sort_values(by=['image'])

In [22]:
y_true = valid_data['bin'].to_numpy()
y_pred = android_data['bin'].to_numpy()

In [23]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_true, y_pred)

0.6011786078998889